<a href="https://colab.research.google.com/github/Muzammil-Elahi/Dividend-Policy-Predictor/blob/main/Model_Training_Top9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import pandas as pd
import numpy as np
import os
import warnings
!pip install optuna
import optuna



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 6.0 MB/s eta 0:00:00


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [16]:
from imblearn.over_sampling import SMOTENC
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, make_scorer, accuracy_score, precision_score, recall_score, f1_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import cross_val_score, GridSearchCV
import pickle

from xgboost import XGBClassifier

In [19]:
from google.colab import drive
drive.mount('/content/drive')

# Import necessary libraries
import pandas as pd

# Define the path to your CSV files directory
csv_directory = '/content/drive/My Drive/path/to/csv/files/'


warnings.filterwarnings('ignore')

X_test_transformed = pd.read_csv('/content/drive/My Drive/X_test_transformed.csv')
y_test = pd.read_csv('/content/drive/My Drive/y_test.csv')
X_train_oversampled = pd.read_csv('/content/drive/My Drive/X_train_oversampled.csv')
y_train_oversampled = pd.read_csv('/content/drive/My Drive/y_train_oversampled.csv')
X_validate_transformed = pd.read_csv('/content/drive/My Drive/X_validate_transformed.csv')
y_validate = pd.read_csv('/content/drive/My Drive/y_validate.csv')



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [24]:
# testing if file read correctly
X_train_oversampled.info()




<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7236 entries, 0 to 7235
Data columns (total 92 columns):
 #   Column                                                Non-Null Count  Dtype  
---  ------                                                --------------  -----  
 0   industry                                              7236 non-null   float64
 1   sector                                                7236 non-null   float64
 2   symbol                                                7236 non-null   float64
 3   year                                                  7236 non-null   float64
 4   adjDividend                                           7236 non-null   float64
 5   dps_growth                                            7236 non-null   float64
 6   dps_growth_rate                                       7236 non-null   float64
 7   interestRate                                          7236 non-null   float64
 8   interestRate_percentage_change                        7236

In [25]:
# test top 9 features
cols = ['dps_growth_rate','dps_growth','interestRate','year','dividendYield','interestRate_percentage_change','freeCashFlowPerShare','adjDividend','debtRatio']
X_train_oversampled = X_train_oversampled[cols]
X_validate_transformed = X_validate_transformed[cols]
X_test_transformed = X_test_transformed[cols]
X_test_transformed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 479 entries, 0 to 478
Data columns (total 9 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   dps_growth_rate                 479 non-null    float64
 1   dps_growth                      479 non-null    float64
 2   interestRate                    479 non-null    float64
 3   year                            479 non-null    float64
 4   dividendYield                   479 non-null    float64
 5   interestRate_percentage_change  479 non-null    float64
 6   freeCashFlowPerShare            479 non-null    float64
 7   adjDividend                     479 non-null    float64
 8   debtRatio                       479 non-null    float64
dtypes: float64(9)
memory usage: 33.8 KB


In [29]:
# Logistic Regression
def objective_function(trial):
    C = trial.suggest_float('C', 0.1, 10, log=True)
    penalty = trial.suggest_categorical('penalty', ['l1', 'l2'])

    model = LogisticRegression(
        C=C,
        penalty=penalty,
        solver='liblinear',
        n_jobs=-1
    )
    metrics = ['roc_auc','accuracy','precison','recall','f1-score']
    # Using cross_val_score to get the average precision score for each fold
    scores = cross_val_score(model, X_train_oversampled, y_train_oversampled, cv=5, scoring='roc_auc')
    roc_auc = np.mean(scores)
    # Printing intermediate results
    print(f"Trial {trial.number}, C: {C}, penalty: {penalty}, ROC-AUC: {roc_auc}")
    return roc_auc


study_lr = optuna.create_study(direction="maximize")
study_lr.optimize(objective_function, n_trials=100)

best_params_lr = study_lr.best_params
print("Best Parameters: ", best_params_lr)
print("Best ROC-AUC Score: ", study_lr.best_value)


[I 2024-05-13 21:25:14,868] A new study created in memory with name: no-name-bdc8fb49-db40-4482-b6c5-10d03cc9a70f
[I 2024-05-13 21:25:16,121] Trial 0 finished with value: 0.7384883491034844 and parameters: {'C': 0.22728087753458193, 'penalty': 'l1'}. Best is trial 0 with value: 0.7384883491034844.


Trial 0, C: 0.22728087753458193, penalty: l1, ROC-AUC: 0.7384883491034844


[I 2024-05-13 21:25:20,459] Trial 1 finished with value: 0.7486069712186458 and parameters: {'C': 2.0739932174785642, 'penalty': 'l1'}. Best is trial 1 with value: 0.7486069712186458.


Trial 1, C: 2.0739932174785642, penalty: l1, ROC-AUC: 0.7486069712186458


[I 2024-05-13 21:25:21,034] Trial 2 finished with value: 0.7404271444435395 and parameters: {'C': 0.269843470387359, 'penalty': 'l1'}. Best is trial 1 with value: 0.7486069712186458.


...

Trial 97, C: 8.796779664172865, penalty: l1, ROC-AUC: 0.749517538754523


[I 2024-05-13 21:28:43,689] Trial 98 finished with value: 0.7447837471037463 and parameters: {'C': 7.66289607442248, 'penalty': 'l2'}. Best is trial 51 with value: 0.7495725349056579.


Trial 98, C: 7.66289607442248, penalty: l2, ROC-AUC: 0.7447837471037463


[I 2024-05-13 21:28:45,363] Trial 99 finished with value: 0.7495209695492351 and parameters: {'C': 8.62580778038581, 'penalty': 'l1'}. Best is trial 51 with value: 0.7495725349056579.


Trial 99, C: 8.62580778038581, penalty: l1, ROC-AUC: 0.7495209695492351
Best Parameters:  {'C': 9.987482428027754, 'penalty': 'l1'}
Best ROC-AUC Score:  0.7495725349056579


In [31]:
# Create and save model
best_model_lr = LogisticRegression(**best_params_lr, solver='liblinear', n_jobs=-1)
with open('best_models_lr_9_features.pkl', 'wb') as file:
    pickle.dump(best_model_lr, file)


In [32]:
# Decision Tree
def objective_function(trial):
    max_depth = trial.suggest_int('max_depth', 1, 10)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)
    criterion = trial.suggest_categorical('criterion', ['gini', 'entropy'])

    model = DecisionTreeClassifier(
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        criterion=criterion
    )
    metrics = ['roc_auc','accuracy','precison','recall','f1-score']
    # Using cross_val_score to get the average precision score for each fold
    scores = cross_val_score(model, X_train_oversampled, y_train_oversampled, cv=5, scoring='roc_auc')
    roc_auc = np.mean(scores)
    # Printing intermediate results
    print(f"Trial {trial.number}, max_depth: {max_depth}, min_samples_split: {min_samples_split}, "
          f"min_samples_leaf: {min_samples_leaf}, criterion: {criterion}, ROC-AUC: {roc_auc}")
    return roc_auc

study_dt = optuna.create_study(direction="maximize")
study_dt.optimize(objective_function, n_trials=100)

best_params_dt = study_dt.best_params
print("Best Parameters: ", best_params_dt)
print("Best ROC-AUC Score: ", study_dt.best_value)


[I 2024-05-13 21:30:21,292] A new study created in memory with name: no-name-1ba150b3-23b5-4dc1-a086-8e7e18c8cca1
[I 2024-05-13 21:30:21,680] Trial 0 finished with value: 0.9281645738027324 and parameters: {'max_depth': 6, 'min_samples_split': 8, 'min_samples_leaf': 3, 'criterion': 'gini'}. Best is trial 0 with value: 0.9281645738027324.


Trial 0, max_depth: 6, min_samples_split: 8, min_samples_leaf: 3, criterion: gini, ROC-AUC: 0.9281645738027324


[I 2024-05-13 21:30:22,093] Trial 1 finished with value: 0.8844063944630551 and parameters: {'max_depth': 5, 'min_samples_split': 9, 'min_samples_leaf': 7, 'criterion': 'entropy'}. Best is trial 0 with value: 0.9281645738027324.


Trial 1, max_depth: 5, min_samples_split: 9, min_samples_leaf: 7, criterion: entropy, ROC-AUC: 0.8844063944630551


[I 2024-05-13 21:30:22,573] Trial 2 finished with value: 0.948657019605464 and parameters: {'max_depth': 8, 'min_samples_split': 9, 'min_samples_leaf': 2, 'criterion': 'entropy'}. Best is trial 2 with value: 0.948657019605464.


...

Trial 97, max_depth: 9, min_samples_split: 4, min_samples_leaf: 6, criterion: entropy, ROC-AUC: 0.9629599386402287


[I 2024-05-13 21:30:58,195] Trial 98 finished with value: 0.9632600463653883 and parameters: {'max_depth': 10, 'min_samples_split': 2, 'min_samples_leaf': 5, 'criterion': 'entropy'}. Best is trial 66 with value: 0.9649359025061974.


Trial 98, max_depth: 10, min_samples_split: 2, min_samples_leaf: 5, criterion: entropy, ROC-AUC: 0.9632600463653883


[I 2024-05-13 21:30:58,556] Trial 99 finished with value: 0.9620179319158921 and parameters: {'max_depth': 9, 'min_samples_split': 3, 'min_samples_leaf': 6, 'criterion': 'entropy'}. Best is trial 66 with value: 0.9649359025061974.


Trial 99, max_depth: 9, min_samples_split: 3, min_samples_leaf: 6, criterion: entropy, ROC-AUC: 0.9620179319158921
Best Parameters:  {'max_depth': 10, 'min_samples_split': 9, 'min_samples_leaf': 6, 'criterion': 'entropy'}
Best ROC-AUC Score:  0.9649359025061974


In [34]:
# Create and save model
best_model_dt = DecisionTreeClassifier(**best_params_dt)
with open('best_models_dt_9_features.pkl', 'wb') as file:
    pickle.dump(best_model_dt, file)

In [35]:
# KNN
def objective_function(trial):
    n_neighbors = trial.suggest_int('n_neighbors', 1, 10)
    weights = trial.suggest_categorical('weights', ['uniform', 'distance'])
    p = trial.suggest_int('p', 1, 5)
    metric = trial.suggest_categorical('metric', ['euclidean', 'manhattan', 'minkowski'])

    model = KNeighborsClassifier(
        n_neighbors=n_neighbors,
        weights=weights,
        p=p,
        metric=metric
    )
    metrics = ['roc_auc','accuracy','precison','recall','f1-score']
    # Using cross_val_score to get the average precision score for each fold
    scores = cross_val_score(model, X_train_oversampled, y_train_oversampled, cv=5, scoring='roc_auc')
    roc_auc = np.mean(scores)
    # Printing intermediate results
    print(f"Trial {trial.number}, n_neighbors: {n_neighbors}, weights: {weights}, p: {p}, metric: {metric}, "
          f"ROC-AUC: {roc_auc}")
    return roc_auc


study_knn = optuna.create_study(direction="maximize")
study_knn.optimize(objective_function, n_trials=100)

best_params_knn = study_knn.best_params
print("Best Parameters: ", best_params_knn)
print("Best ROC-AUC Score: ", study_knn.best_value)

[I 2024-05-13 21:31:48,347] A new study created in memory with name: no-name-74601f39-9752-4437-8490-cbb2f4dae9d4
[I 2024-05-13 21:31:48,608] Trial 0 finished with value: 0.9262938956488721 and parameters: {'n_neighbors': 2, 'weights': 'distance', 'p': 5, 'metric': 'manhattan'}. Best is trial 0 with value: 0.9262938956488721.


Trial 0, n_neighbors: 2, weights: distance, p: 5, metric: manhattan, ROC-AUC: 0.9262938956488721


[I 2024-05-13 21:31:48,857] Trial 1 finished with value: 0.934229249407345 and parameters: {'n_neighbors': 9, 'weights': 'distance', 'p': 2, 'metric': 'minkowski'}. Best is trial 1 with value: 0.934229249407345.


Trial 1, n_neighbors: 9, weights: distance, p: 2, metric: minkowski, ROC-AUC: 0.934229249407345


[I 2024-05-13 21:31:49,101] Trial 2 finished with value: 0.9262938956488721 and parameters: {'n_neighbors': 2, 'weights': 'distance', 'p': 2, 'metric': 'manhattan'}. Best is trial 1 with value: 0.934229249407345.


...

Trial 97, n_neighbors: 10, weights: distance, p: 2, metric: minkowski, ROC-AUC: 0.9340136312688024


[I 2024-05-13 21:32:22,289] Trial 98 finished with value: 0.9490057819735818 and parameters: {'n_neighbors': 9, 'weights': 'distance', 'p': 4, 'metric': 'manhattan'}. Best is trial 36 with value: 0.9490860153904788.


Trial 98, n_neighbors: 9, weights: distance, p: 4, metric: manhattan, ROC-AUC: 0.9490057819735818


[I 2024-05-13 21:32:22,578] Trial 99 finished with value: 0.9402317057740323 and parameters: {'n_neighbors': 10, 'weights': 'uniform', 'p': 3, 'metric': 'manhattan'}. Best is trial 36 with value: 0.9490860153904788.


Trial 99, n_neighbors: 10, weights: uniform, p: 3, metric: manhattan, ROC-AUC: 0.9402317057740323
Best Parameters:  {'n_neighbors': 10, 'weights': 'distance', 'p': 3, 'metric': 'manhattan'}
Best ROC-AUC Score:  0.9490860153904788


In [36]:
# Create and save model
best_model_knn = KNeighborsClassifier(**best_params_knn)
with open('best_models_knn_9_features.pkl', 'wb') as file:
    pickle.dump(best_model_knn, file)

In [38]:

# Random Forest
def objective_function(trial):
    n_estimators = trial.suggest_int('n_estimators', 2, 50)
    max_depth = trial.suggest_int('max_depth', 1, 10)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)

    model = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        n_jobs=-1
    )

    # Using cross_val_score to get the average ROC-AUC score for each fold
    scores = cross_val_score(model, X_train_oversampled, y_train_oversampled, cv=5, scoring='roc_auc')
    roc_auc = np.mean(scores)
    # Printing intermediate results
    print(f"Trial {trial.number}, n_estimators: {n_estimators}, max_depth: {max_depth}, "
          f"min_samples_split: {min_samples_split}, min_samples_leaf: {min_samples_leaf}, ROC-AUC: {roc_auc}")
    return roc_auc

study_rf = optuna.create_study(direction="maximize")
study_rf.optimize(objective_function, n_trials=100)

best_params_rf = study_rf.best_params
print("Best Parameters: ", best_params_rf)
print("Best ROC-AUC: Score: ", study_rf.best_value)

[I 2024-05-13 21:48:30,811] A new study created in memory with name: no-name-6300bfdf-14b7-47b1-91f1-cb2f0c612e95
[I 2024-05-13 21:48:31,187] Trial 0 finished with value: 0.9216865307094599 and parameters: {'n_estimators': 5, 'max_depth': 4, 'min_samples_split': 2, 'min_samples_leaf': 5}. Best is trial 0 with value: 0.9216865307094599.


Trial 0, n_estimators: 5, max_depth: 4, min_samples_split: 2, min_samples_leaf: 5, ROC-AUC: 0.9216865307094599


[I 2024-05-13 21:48:31,657] Trial 1 finished with value: 0.936748302640571 and parameters: {'n_estimators': 8, 'max_depth': 4, 'min_samples_split': 5, 'min_samples_leaf': 8}. Best is trial 1 with value: 0.936748302640571.


Trial 1, n_estimators: 8, max_depth: 4, min_samples_split: 5, min_samples_leaf: 8, ROC-AUC: 0.936748302640571


[I 2024-05-13 21:48:32,470] Trial 2 finished with value: 0.9192624544444714 and parameters: {'n_estimators': 23, 'max_depth': 3, 'min_samples_split': 6, 'min_samples_leaf': 5}. Best is trial 1 with value: 0.936748302640571.


...

Trial 97, n_estimators: 42, max_depth: 9, min_samples_split: 4, min_samples_leaf: 2, ROC-AUC: 0.9863763239616752


[I 2024-05-13 21:51:52,088] Trial 98 finished with value: 0.9702672796479874 and parameters: {'n_estimators': 44, 'max_depth': 6, 'min_samples_split': 3, 'min_samples_leaf': 2}. Best is trial 92 with value: 0.9888881828695804.


Trial 98, n_estimators: 44, max_depth: 6, min_samples_split: 3, min_samples_leaf: 2, ROC-AUC: 0.9702672796479874


[I 2024-05-13 21:51:54,107] Trial 99 finished with value: 0.9884081803339267 and parameters: {'n_estimators': 41, 'max_depth': 10, 'min_samples_split': 4, 'min_samples_leaf': 1}. Best is trial 92 with value: 0.9888881828695804.


Trial 99, n_estimators: 41, max_depth: 10, min_samples_split: 4, min_samples_leaf: 1, ROC-AUC: 0.9884081803339267
Best Parameters:  {'n_estimators': 46, 'max_depth': 10, 'min_samples_split': 3, 'min_samples_leaf': 1}
Best ROC-AUC: Score:  0.9888881828695804


In [40]:
# Create and save model
best_model_rf = RandomForestClassifier(**best_params_rf, n_jobs=-1)
with open('best_models_rf_9_features.pkl', 'wb') as file:
    pickle.dump(best_model_rf, file)

label_encoder = LabelEncoder()
# Fit the encoder and transform the target variable
y_train_oversampled_encoded = label_encoder.fit_transform(y_train_oversampled)
# This suppresses printing logs
optuna.logging.set_verbosity(optuna.logging.WARNING)

In [41]:
label_encoder = LabelEncoder()
# Fit the encoder and transform the target variable
y_train_oversampled_encoded = label_encoder.fit_transform(y_train_oversampled)
# This suppresses printing logs
optuna.logging.set_verbosity(optuna.logging.WARNING)

def objective_function(trial):
    n_estimators = trial.suggest_int('n_estimators', 2, 50)
    max_depth = trial.suggest_int('max_depth', 1, 10)
    learning_rate = trial.suggest_float('learning_rate', 0.001, 0.9, log=True)
    min_child_weight = trial.suggest_int('min_child_weight', 1, 10)
    subsample = trial.suggest_float('subsample', 0.5, 1.0)
    colsample_bytree = trial.suggest_float('colsample_bytree', 0.5, 1.0)
    gamma = trial.suggest_float('gamma', 0, 1.0)
    reg_alpha = trial.suggest_float('reg_alpha', 0, 1)
    reg_lambda = trial.suggest_float('reg_lambda', 0, 1)

    model = XGBClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        learning_rate=learning_rate,
        min_child_weight=min_child_weight,
        subsample=subsample,
        colsample_bytree=colsample_bytree,
        gamma=gamma,
        reg_alpha=reg_alpha,
        reg_lambda=reg_lambda,
        use_label_encoder=False,
        n_jobs=-1
    )

    # Using cross_val_score to get the average ROC-AUC score for each fold
    scores = cross_val_score(model, X_train_oversampled, y_train_oversampled_encoded, cv=5, scoring='roc_auc')
    roc_auc = np.mean(scores)
    # Printing intermediate results
    print(f"Trial {trial.number}, n_estimators: {n_estimators}, max_depth: {max_depth}, learning_rate: {learning_rate},"
          f"min_child_weight: {min_child_weight}, subsample: {subsample}, colsample_bytree: {colsample_bytree}, "
          f"gamma: {gamma}, reg_alpha: {reg_alpha}, reg_lambda: {reg_lambda}, ROC-AUC: {roc_auc}")
    return roc_auc


study_xgb = optuna.create_study(direction="maximize")
study_xgb.optimize(objective_function, n_trials=100)

best_params_xgb = study_xgb.best_params
print("Best Parameters: ", best_params_xgb)
print("Best ROC-AUC Score: ", study_xgb.best_value)

best_model_xgb = XGBClassifier(**best_params_xgb, use_label_encoder=False, n_jobs=-1)
with open('best_models_xgb_9_features.pkl', 'wb') as file:
    pickle.dump(best_model_xgb, file)


Trial 0, n_estimators: 36, max_depth: 3, learning_rate: 0.004319387223562214,min_child_weight: 6, subsample: 0.990328144354381, colsample_bytree: 0.7049814464590884, gamma: 0.10689411429280582, reg_alpha: 0.34716903645578956, reg_lambda: 0.8986552674694452, ROC-AUC: 0.9162646594306398
Trial 1, n_estimators: 11, max_depth: 2, learning_rate: 0.003435431435329658,min_child_weight: 7, subsample: 0.704812756869506, colsample_bytree: 0.5406827275992673, gamma: 0.43241203367026515, reg_alpha: 0.01011455339208911, reg_lambda: 0.41558177488256065, ROC-AUC: 0.8600690146495833
Trial 2, n_estimators: 47, max_depth: 4, learning_rate: 0.004584903404826095,min_child_weight: 4, subsample: 0.7747186294038515, colsample_bytree: 0.6263304686156592, gamma: 0.5699253522578187, reg_alpha: 0.4007507877207517, reg_lambda: 0.6635227179441499, ROC-AUC: 0.9524717472129239
Trial 3, n_estimators: 17, max_depth: 4, learning_rate: 0.030103760648190973,min_child_weight: 4, subsample: 0.5907478879898802, colsample_byt

In [42]:
# Model selection - Compare Performance
with open('best_models_lr_9_features.pkl', 'rb') as file:
    best_model_lr = pickle.load(file)
with open('best_models_dt_9_features.pkl', 'rb') as file:
    best_model_dt = pickle.load(file)
with open('best_models_knn_9_features.pkl', 'rb') as file:
    best_model_knn = pickle.load(file)
with open('best_models_rf_9_features.pkl', 'rb') as file:
    best_model_rf = pickle.load(file)
with open('best_models_xgb_9_features.pkl', 'rb') as file:
    best_model_xgb = pickle.load(file)

print("Testing Performances...Please wait")
best_model_lr.fit(X_train_oversampled, y_train_oversampled)
predicted_probs = best_model_lr.predict_proba(X_test_transformed)[:, 1]
lr_performance = roc_auc_score(y_test, predicted_probs)

best_model_dt.fit(X_train_oversampled, y_train_oversampled)
predicted_probs = best_model_dt.predict_proba(X_test_transformed)[:, 1]
dt_performance = roc_auc_score(y_test, predicted_probs)

best_model_knn.fit(X_train_oversampled, y_train_oversampled)
predicted_probs = best_model_knn.predict_proba(X_test_transformed)[:, 1]
knn_performance = roc_auc_score(y_test, predicted_probs)

best_model_rf.fit(X_train_oversampled, y_train_oversampled)
predicted_probs = best_model_rf.predict_proba(X_test_transformed)[:, 1]
rf_performance = roc_auc_score(y_test, predicted_probs)

best_model_xgb.fit(X_train_oversampled, y_train_oversampled_encoded)
predicted_probs = best_model_xgb.predict_proba(X_test_transformed)[:, 1]
xgb_performance = roc_auc_score(y_test, predicted_probs)

# Test performance of the models are
print(f"Logistic Regression Test ROCAUC: {lr_performance}")
print(f"Decision Tree Test ROCAUC: {dt_performance}")
print(f"KNN Test ROCAUC: {knn_performance}")
print(f"Random Forest Test ROCAUC: {rf_performance}")
print(f"XGBoost Test ROCAUC: {xgb_performance}")

Testing Performances...Please wait
Logistic Regression Test ROCAUC: 0.7553113553113553
Decision Tree Test ROCAUC: 0.5163003663003662
KNN Test ROCAUC: 0.7565934065934066
Random Forest Test ROCAUC: 0.7597985347985348
XGBoost Test ROCAUC: 0.8017399267399268
